In [121]:
import pandas as pd
import numpy as np
import torch

# Data

# Model

In [91]:
grouped_df = pd.read_csv("data/skylab_instagram_datathon_dataset_cleaned.csv", sep=",").drop(["index"], axis=1)

In [92]:
grouped_df.columns

Index(['period_end_date', 'business_entity_doing_business_as_name',
       'followers', 'pictures', 'videos', 'comments', 'likes'],
      dtype='object')

In [130]:
def generate_data(df, sequence_length=30, K=4):
    label = np.empty((0, 0, 0)

    for col_val in df["business_entity_doing_business_as_name"].unique():
        df_filt = df[df["business_entity_doing_business_as_name"] == col_val]
        # Drop useless columns
        values = df_filt.drop(["business_entity_doing_business_as_name", "period_end_date"], axis=1).values
        # For now cut-out first elements to have a perfect divisor
        pred_val = torch.tensor(values[-K:]).unsqueeze(dim=0)
        values = values[(len(values) - K)%sequence_length:len(values) - K]
        # List of df split per sequence_length
        values_list = torch.tensor(np.array(np.split(values, values.shape[0] // sequence_length)))
        print(values_list.unsqueeze(dim=0).shape)

        print(pred_val.shape)
        break

In [131]:
generate_data(grouped_df)


torch.Size([1, 10, 30, 5])
torch.Size([1, 4, 5])


In [107]:
generate_data(grouped_df)

(0, 1)


In [91]:
import torch
import torch.nn.functional as F
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

/home/ggil/anaconda3/envs/hackathon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [92]:
# Set seeds and device
seed = 2
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cpu


## Load dataset

In [ ]:
from utils.dataset import get_datasets

DATASET = "cifar10"
path = "/kaggle/input/utkface-new/UTKFace"

train_dataset, val_dataset = get_datasets(path, test_size=0.1, augment=True)

In [ ]:
from machine_unlearning.experiments import Experiment

model = Experiment(cfg)

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, Callback, EarlyStopping

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='/kaggle/working/',
    filename='{epoch:02d}-{val_loss:.2f}'
)

class PrintCallback(pl.Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        print(f"Training loss: {trainer.callback_metrics['train_loss']}")
        
    def on_validation_end(self, trainer, pl_module):
        print(f"Validation loss: {trainer.callback_metrics['val_loss']}, Accuracy: {trainer.callback_me

In [ ]:
trainer.fit(model, cfg["data"], cfg["val_data"])